In [1]:
train_dir = "/kaggle/input/compcars431modelstraintest/Compcars431ModelsTrainTest/train"
test_dir = "/kaggle/input/compcars431modelstraintest/Compcars431ModelsTrainTest/val"

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, InputLayer, BatchNormalization, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

2024-08-17 03:32:26.398801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 03:32:26.398905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 03:32:26.524584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
img_width, img_height = 400, 400
batch_size = 16
epochs = 100
num_classes = 431

In [4]:
train_datagen = ImageDataGenerator( rescale=1./255, rotation_range=25, width_shift_range=0.25, 
                                   height_shift_range=0.25, shear_range=0.25, zoom_range=0.25, 
                                   horizontal_flip=True, fill_mode='nearest' )
        
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory( train_dir, 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, class_mode='categorical', 
                                                    shuffle=True)

test_generator = test_datagen.flow_from_directory( test_dir, 
                                                  target_size=(img_width, img_height), 
                                                  batch_size=batch_size, class_mode='categorical', 
                                                  shuffle=False )

Found 36334 images belonging to 431 classes.
Found 15749 images belonging to 431 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model

from tensorflow.keras.layers import *

In [6]:
num_heads = 5
reduction_ratio = 8
dropout_rate = 0.02
l2_reg = 0.001
simam_lambda = 0.0001

In [7]:
class SimAM(tf.keras.layers.Layer):
    def __init__(self, eps=0.0001, activation=tf.nn.sigmoid, trainable=True, name=None, **kwargs):
        super(SimAM, self).__init__(name=name, trainable=trainable, **kwargs)
        self.activation = activation
        self.eps = eps

    def build(self, input_shape):
        self.height, self.width, self.channels = input_shape[1], input_shape[2], input_shape[3]
        self.norm = 4. / (self.height * self.width - 1)
        super().build(input_shape)

    def call(self, inputs, **kwargs):
        minus_mu_square = tf.square(inputs - tf.reduce_mean(inputs, axis=(1, 2), keepdims=True))
        out = minus_mu_square / tf.maximum(
            tf.reduce_sum(minus_mu_square, axis=(1, 2), keepdims=True) * self.norm,
            self.eps) + 0.5
        return inputs * self.activation(out)

In [8]:
class CSA(layers.Layer):
    def __init__(self, num_heads, channels, dropout_rate=0.02,l2_reg=0.001, **kwargs):
        super(CSA, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.head_channels = channels // num_heads
        self.heads = []

        for _ in range(num_heads):
            self.heads.append(self.build_SA_block(channels=self.head_channels,
                                                              dropout_rate=dropout_rate,
                                                              l2_reg=l2_reg))

        self.simam = SimAM(eps=0.0001, activation=tf.nn.sigmoid)
        self.dropout = layers.Dropout(dropout_rate)
        self.batch_norm = layers.BatchNormalization()

    def build_SA_block(self, channels, dropout_rate, l2_reg):
        input_tensor = tf.keras.Input(shape=(6, 6, channels))

        # Channel-wise attention Global features
        input_channels = channels

        conv_before_attention0 = DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(input_tensor)

        conv_before_attention1 = DepthwiseConv2D(kernel_size=(1, 2), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention0)

        conv_before_attention2 = DepthwiseConv2D(kernel_size=(2, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention0)

        conv_before_attention3 = Multiply()([conv_before_attention1, conv_before_attention2])

        conv_before_attention4 = DepthwiseConv2D(kernel_size=(1, 3), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention3)

        conv_before_attention5 = DepthwiseConv2D(kernel_size=(3, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention3)

        conv_before_attention6 = Multiply()([conv_before_attention4, conv_before_attention5])

        conv_before_attention7 = DepthwiseConv2D(kernel_size=(1, 4), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention6)

        conv_before_attention8 = DepthwiseConv2D(kernel_size=(4, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention6)

        conv_before_attention9 = Multiply()([conv_before_attention7, conv_before_attention8])

        conv_before_attention10 = DepthwiseConv2D(kernel_size=(1, 5), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention9)

        conv_before_attention11 = DepthwiseConv2D(kernel_size=(5, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(conv_before_attention9)

        conv_before_attention12 = Multiply()([conv_before_attention10, conv_before_attention11])

        conv_before_attention12 = Dropout(dropout_rate)(conv_before_attention12)

        avg_pool = GlobalAveragePooling2D()(conv_before_attention12)
        max_pool = GlobalMaxPooling2D()(conv_before_attention12)
        concat = Concatenate()([avg_pool, max_pool])

        sequential_attention = Reshape((1, 1, concat.shape[-1]))(concat)
        sequential_attention = Lambda(lambda x: tf.keras.backend.repeat_elements(x, rep=6, axis=1))(sequential_attention)
        sequential_attention = Lambda(lambda x: tf.keras.backend.repeat_elements(x, rep=6, axis=2))(sequential_attention)

        sequential_attention0 = DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                                       depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                       padding='same')(sequential_attention)

        sequential_attention1 = DepthwiseConv2D(kernel_size=(1, 2), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention0)

        sequential_attention2 = DepthwiseConv2D(kernel_size=(2, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention0)

        sequential_attention3 = Multiply()([sequential_attention1, sequential_attention2])

        sequential_attention4 = DepthwiseConv2D(kernel_size=(1, 3), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention3)

        sequential_attention5 = DepthwiseConv2D(kernel_size=(3, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention3)

        sequential_attention6 = Multiply()([sequential_attention4, sequential_attention5])

        sequential_attention7 = DepthwiseConv2D(kernel_size=(1, 4), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention6)

        sequential_attention8 = DepthwiseConv2D(kernel_size=(4, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention6)

        sequential_attention9 = Multiply()([sequential_attention7, sequential_attention8])

        sequential_attention10 = DepthwiseConv2D(kernel_size=(1, 5), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention9)

        sequential_attention11 = DepthwiseConv2D(kernel_size=(5, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention9)

        sequential_attention12 = Multiply()([sequential_attention10, sequential_attention11])

        sequential_attention12 = Dropout(dropout_rate)(sequential_attention12)

        sequential_attention13 = Concatenate()([conv_before_attention12, sequential_attention12])

        # Row-wise attention
        row_attention0 = DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(conv_before_attention12)

        row_attention1 = DepthwiseConv2D(kernel_size=(1, 2), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention0)

        row_attention2 = DepthwiseConv2D(kernel_size=(1, 3), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention1)

        row_attention3 = DepthwiseConv2D(kernel_size=(1, 4), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention2)

        row_attention4 = DepthwiseConv2D(kernel_size=(1, 5), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention3)

        row_attention4 = Dropout(dropout_rate)(row_attention4)

        row_attention5 = Concatenate()([sequential_attention13, row_attention4])

        # Column-wise attention
        col_attention0 = DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(conv_before_attention12)

        col_attention1 = DepthwiseConv2D(kernel_size=(2, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention0)

        col_attention2 = DepthwiseConv2D(kernel_size=(3, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention1)

        col_attention3 = DepthwiseConv2D(kernel_size=(4, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention2)

        col_attention4 = DepthwiseConv2D(kernel_size=(5, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention3)

        col_attention4 = Dropout(dropout_rate)(col_attention4)

        col_attention5 = Concatenate()([sequential_attention13, col_attention4])

        # Combine row and column attentions
        sequential_attention14 = Multiply()([row_attention5, col_attention5])
        sequential_attention14 = Dropout(dropout_rate)(sequential_attention14)

        conv_after_attention0 = DepthwiseConv2D( kernel_size=(1, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(sequential_attention14)

        conv_after_attention1 = DepthwiseConv2D( kernel_size=(1, 2), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention0)

        conv_after_attention2 = DepthwiseConv2D( kernel_size=(2, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention0)

        conv_after_attention3 = Multiply()([conv_after_attention1, conv_after_attention2])

        conv_after_attention4 = DepthwiseConv2D( kernel_size=(1, 3), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention3)

        conv_after_attention5 = DepthwiseConv2D( kernel_size=(3, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention3)

        conv_after_attention6 = Multiply()([conv_after_attention4, conv_after_attention5])

        conv_after_attention7 = DepthwiseConv2D( kernel_size=(1, 4), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention6)

        conv_after_attention8 = DepthwiseConv2D( kernel_size=(4, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention6)

        conv_after_attention9 = Multiply()([conv_after_attention7, conv_after_attention8])

        conv_after_attention10 = DepthwiseConv2D( kernel_size=(1, 5), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention9)

        conv_after_attention11 = DepthwiseConv2D( kernel_size=(5, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention9)

        conv_after_attention12 = Multiply()([conv_after_attention10,
                                                     conv_after_attention11])

        conv_after_attention12 = Dropout(dropout_rate)(conv_after_attention12)

        conv_after_attention12 = Concatenate()([conv_before_attention12, conv_after_attention12])

        # Channel-wise attention local features
        sequential_attention15 = DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                                       depthwise_initializer='he_normal',
                                       depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                       padding='same')(conv_after_attention12)

        sequential_attention16 = DepthwiseConv2D(kernel_size=(1, 2), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention15)

        sequential_attention17 = DepthwiseConv2D(kernel_size=(2, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention15)

        sequential_attention18 = Multiply()([sequential_attention16, sequential_attention17])

        sequential_attention19 = DepthwiseConv2D(kernel_size=(1, 3), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention18)

        sequential_attention20 = DepthwiseConv2D(kernel_size=(3, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention18)

        sequential_attention21 = Multiply()([sequential_attention19, sequential_attention20])

        sequential_attention22 = DepthwiseConv2D(kernel_size=(1, 4), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention21)

        sequential_attention23 = DepthwiseConv2D(kernel_size=(4, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention22)

        sequential_attention24 = Multiply()([sequential_attention22, sequential_attention23])

        sequential_attention25 = DepthwiseConv2D(kernel_size=(1, 5), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention24)

        sequential_attention26 = DepthwiseConv2D(kernel_size=(5, 1), activation='sigmoid',
                                        depthwise_initializer='he_normal',
                                        depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                        padding='same')(sequential_attention24)

        sequential_attention27 = Multiply()([sequential_attention25, sequential_attention26])

        sequential_attention27 = Dropout(dropout_rate)(sequential_attention27)

        sequential_attention28 = Concatenate()([conv_before_attention12, sequential_attention27])

        # Row-wise attention
        row_attention6= DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(conv_before_attention12)

        row_attention7= DepthwiseConv2D(kernel_size=(1, 2), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention6)

        row_attention8= DepthwiseConv2D(kernel_size=(1, 3), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention7)

        row_attention9= DepthwiseConv2D(kernel_size=(1, 4), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention8)

        row_attention10= DepthwiseConv2D(kernel_size=(1, 5), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(row_attention9)

        row_attention10 = Dropout(dropout_rate)(row_attention10)
        row_attention11 = Concatenate()([sequential_attention28, row_attention10])

        # Column-wise attention
        col_attention6= DepthwiseConv2D(kernel_size=(1, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(conv_before_attention12)

        col_attention7= DepthwiseConv2D(kernel_size=(2, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention6)

        col_attention8= DepthwiseConv2D(kernel_size=(3, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention7)

        col_attention9= DepthwiseConv2D(kernel_size=(4, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention8)

        col_attention10= DepthwiseConv2D(kernel_size=(5, 1), activation='sigmoid',
                               depthwise_initializer='he_normal', depthwise_regularizer=regularizers.l2(l2_reg),
                               padding='same')(col_attention9)

        col_attention10= Dropout(dropout_rate)(col_attention10)
        col_attention11= Concatenate()([sequential_attention28, col_attention10])

        # Combine row and column attentions
        sequential_attention29=Multiply()([row_attention11, col_attention11])
        sequential_attention29=Dropout(dropout_rate)(sequential_attention29)

        conv_after_attention13 = DepthwiseConv2D( kernel_size=(1, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(sequential_attention29)

        conv_after_attention14= DepthwiseConv2D( kernel_size=(1, 2), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention13)

        conv_after_attention15= DepthwiseConv2D( kernel_size=(2, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention13)

        conv_after_attention16=Multiply()([conv_after_attention14, conv_after_attention15])

        conv_after_attention17= DepthwiseConv2D( kernel_size=(1, 3), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention16)

        conv_after_attention18= DepthwiseConv2D( kernel_size=(3, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention16)

        conv_after_attention19=Multiply()([conv_after_attention17, conv_after_attention18])

        conv_after_attention20= DepthwiseConv2D( kernel_size=(1, 4), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention19)

        conv_after_attention21= DepthwiseConv2D( kernel_size=(4, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention19)

        conv_after_attention22=Multiply()([conv_after_attention20, conv_after_attention21])

        conv_after_attention23= DepthwiseConv2D( kernel_size=(1, 5), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention22)

        conv_after_attention24= DepthwiseConv2D( kernel_size=(5, 1), activation='sigmoid',
                                            depthwise_initializer='he_normal',
                                              depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
                                            padding='same')(conv_after_attention23)

        conv_after_attention25=Multiply()([conv_after_attention23,
                                                   conv_after_attention24])

        conv_after_attention25= Dropout(dropout_rate)(conv_after_attention25)

        conv_after_attention25 = Concatenate()([conv_before_attention12, conv_after_attention25])
        
        dual_sequential_attention = Concatenate()([conv_after_attention12, conv_after_attention25])

        dual_sequential_attention = BatchNormalization()(dual_sequential_attention)
        dual_sequential_attention = Dropout(dropout_rate)(dual_sequential_attention)

        return tf.keras.Model(inputs=input_tensor, outputs=dual_sequential_attention)

    def call(self, inputs):
        head_outputs = []

        for i in range(self.num_heads):
            head_input = inputs[:, :, :, i * self.head_channels:(i + 1) * self.head_channels]
            head_output = self.heads[i](head_input)
            head_outputs.append(head_output)

        cbam_attention = Concatenate()(head_outputs)
        simam_attention = self.simam(inputs)

        fused_attention = tf.concat([cbam_attention, simam_attention], axis=-1)
        fused_attention = self.dropout(fused_attention)
        fused_attention = self.batch_norm(fused_attention)

        return fused_attention

In [9]:
from tensorflow.keras.layers import Lambda, DepthwiseConv2D, BatchNormalization, Dropout, Reshape, UpSampling2D, Multiply

In [10]:
from tensorflow.keras.applications import DenseNet201

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
x = base_model.output
x = BatchNormalization()(x)
x = Dropout(0.05)(x)
print(x.shape)

x = layers.AveragePooling2D(pool_size=2)(x)
x = BatchNormalization()(x)
x = Dropout(0.05)(x)
print(x.shape)

x = SeparableConv2D( 512, kernel_size=(5, 5), activation='sigmoid',
            depthwise_initializer='he_normal', pointwise_initializer='he_normal',
            depthwise_regularizer=tf.keras.regularizers.l2(l2_reg),
            pointwise_regularizer=tf.keras.regularizers.l2(l2_reg),
            padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.05)(x)
print(x.shape)

x = CSA(num_heads, 512, dropout_rate, l2_reg)(x)
x = BatchNormalization()(x)
x = Dropout(0.05)(x)
print("attention",x.shape)

x = layers.GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.05)(x)
print(x.shape)

preds = Dense(units=196, activation='softmax')(x)
model1 = tf.keras.Model(inputs=base_model.input, outputs=preds)

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(None, 12, 12, 1920)
(None, 6, 6, 1920)
(None, 6, 6, 512)
attention (None, 6, 6, 7142)
(None, 7142)


In [11]:
for layer in model1.layers:
    layer.trainable = True

In [12]:
model1.load_weights("/kaggle/input/acc97-50-ep25-compcars431-dr-0-2/CompCars431_(dr_0.2)_Acc97.50_ep25_DenseNet201_Modified_MHSimamCBAM_Compcars431.h5")

In [12]:
model1.compile(optimizer=Adam(learning_rate = 0.00001), loss = 'categorical_crossentropy', 
              metrics=['accuracy','mse'])

checkpoint = ModelCheckpoint('DenseNet201_Modified_MHSimamCBAM_Compcars431.weights.h5', monitor='val_accuracy', 
                             save_best_only = True, save_weights_only = True, mode ='max', verbose = 2)

early_stop = EarlyStopping(monitor = 'val_accuracy', patience = 8, mode = 'max', verbose = 2)

reduce_learning_rate=ReduceLROnPlateau(monitor = "val_accuracy", factor = 0.1, patience = 4, verbose = 2)

In [13]:
total_params = model1.count_params()
print(f"Total parameters in the model: {total_params}")

Total parameters in the model: 21292216


In [14]:
trainable_params = tf.reduce_sum([tf.reduce_prod(v.shape) for v in model1.trainable_variables])
print(f"Trainable parameters in the model: {trainable_params}")

Trainable parameters in the model: 20998344


In [16]:
non_trainable_params = tf.reduce_sum([tf.reduce_prod(v.shape) for v in model1.non_trainable_weights])
print(f"Non-trainable parameters in the model: {non_trainable_params}")

Non-trainable parameters in the model: 293872


In [17]:
import time
start_time = time.time()

history=model1.fit(train_generator, epochs = 50, validation_data = test_generator, 
                  callbacks = [checkpoint, early_stop, reduce_learning_rate])

end_time = time.time()
preprocessing_time = end_time - start_time

print("Preprocessing completed. Time taken:", preprocessing_time, "seconds")

Epoch 1/50


2023-12-10 01:36:22.402123: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_5/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2271/2271 [==============================] - ETA: 0s - loss: 0.1347 - accuracy: 0.9855 - mse: 5.1599e-05
Epoch 1: val_accuracy improved from -inf to 0.97663, saving model to DenseNet201_Modified_MHSimamCBAM_Compcars431.h5
2271/2271 [==============================] - 2168s 841ms/step - loss: 0.1347 - accuracy: 0.9855 - mse: 5.1599e-05 - val_loss: 0.1869 - val_accuracy: 0.9766 - val_mse: 8.4892e-05 - lr: 1.0000e-05
Epoch 2/50
2271/2271 [==============================] - ETA: 0s - loss: 0.1186 - accuracy: 0.9887 - mse: 3.8641e-05
Epoch 2: val_accuracy improved from 0.97663 to 0.97790, saving model to DenseNet201_Modified_MHSimamCBAM_Compcars431.h5
2271/2271 [==============================] - 1617s 712ms/step - loss: 0.1186 - accuracy: 0.9887 - mse: 3.8641e-05 - val_loss: 0.1799 - val_accuracy: 0.9779 - val_mse: 7.9174e-05 - lr: 1.0000e-05
Epoch 3/50
2271/2271 [==============================] - ETA: 0s - loss: 0.1102 - accuracy: 0.9906 - mse: 3.2265e-05
Epoch 3: val_accuracy improved from 

KeyboardInterrupt: 

In [18]:
import IPython
IPython.display.FileLink('DenseNet201_Modified_MHSimamCBAM_Compcars431.weights.h5')

/kaggle/working/DenseNet201_Modified_MHSimamCBAM_Compcars431.h5